In [ ]:
!pip install sentence-transformers openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/YOLO/fiqa.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df.head()

,query_id,doc_id,relevance,query_text,document_text
0,0,18850,1,What is considered a business expense on a bus...,The IRS Guidance pertaining to the subject. I...
1,4,196463,1,Business Expense - Car Insurance Deductible Fo...,"As a general rule, you must choose between a m..."
2,5,69306,1,Starting a new online business,Most US states have rules that go something li...
3,6,560251,1,“Business day” and “due date” for bills,I don't believe Saturday is a business day eit...
4,6,188530,1,“Business day” and “due date” for bills,You definitely have an argument for getting th...


In [ ]:
from transformers import BertTokenizer

# Initialize the tokenizer (example with BERT)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Shorten each passage to fit within 512 tokens
def truncate_passages(passages, max_length=512):
    shortened_passages = []
    for passage in passages:
        # Tokenize and truncate the passage
        tokens = tokenizer.tokenize(passage)
        if len(tokens) > max_length:
            tokens = tokens[:max_length]
        shortened_passage = tokenizer.convert_tokens_to_string(tokens)
        shortened_passages.append(shortened_passage)
    return shortened_passages

# Example usage
corpus = [str(doc) for doc in df['document_text'].dropna()]
passages = truncate_passages(corpus)
print(passages)

In [ ]:
passages[1]

"as a general rule , you must choose between a mileage deduction or an actual expenses deduction . the idea is that the mileage deduction is supposed to cover all costs of using the car . exceptions include parking fees and tolls , which can be deducted separately under either method . you explicitly cannot deduct insurance costs if you claim a mileage deduction . separately , you probably won ' t be able to deduct the deductible for your car as a casualty loss . you first subtract $ 100 from the deductible and then divide it by your adjusted gross income ( agi ) from your tax return . if your deductible is over 10 % of your agi , you can deduct it . note that even with a $ 1500 deductible , you won ' t be able to deduct anything if you made more than $ 14 , 000 for the year . for most people , the insurance deductible just isn ' t large enough relative to income to be tax deductible . source"

# ***Small Embedding Model***

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

# Example query and passages
query = "What is inflation?"

# Load pre-trained embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode the query and passages
query_embedding = embedding_model.encode(query, convert_to_tensor=True)
passage_embeddings = embedding_model.encode(passages, convert_to_tensor=True)

# Compute cosine similarities between the query and all passages
cosine_scores = util.pytorch_cos_sim(query_embedding, passage_embeddings)

# Get the top-k passages (let's choose k=10)
top_k = 10
top_results = torch.topk(cosine_scores, k=top_k)

# Extract the top-k passage indices and their corresponding similarities
top_k_indices = top_results.indices.squeeze().tolist()
top_k_similarities = top_results.values.squeeze().tolist()

# Store top-k passages and their scores
retrieved_passages = [(passages[idx], top_k_similarities[i]) for i, idx in enumerate(top_k_indices)]

# Output the top-k passages and their similarity scores
for passage, score in retrieved_passages:
    print(f"Passage: {passage}, Score: {score}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Passage: inflation refers to the money supply . think of all money being air in a balloon . inflation is what happens when you blow more air in the balloon . deflation is what happens when you let air escape . inflation may cause prices to go up . however there are many scenarios possible in which this does not happen . for example , at the same time of inflation , there might be unemployment , making consumers unable to pay higher prices . or some important resource ( oil ) may go down in price ( due to political reasons , war has ended etc ) , compensating for the money having less value . similarly , peoples wages will tend to rise over time . they have to , otherwise everyone would be earning less , due to inflation . however again there are many scenarios in which wages do not keep up with inflation , or rise much faster . in fact over the past 40 years or so , us wages have not been able to keep up with inflation , making the average worker ' poorer ' than 40 years ago . at its c

# ***Small Embedding Model + Ranking Model***

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Initialize the tokenizer and model (you can use any ranking model fine-tuned for Q&A tasks)
tokenizer = BertTokenizer.from_pretrained('cross-encoder/ms-marco-MiniLM-L-12-v2')
model = BertForSequenceClassification.from_pretrained('cross-encoder/ms-marco-MiniLM-L-12-v2')

# Function to calculate relevance score using the ranking model
def rank_passages(query, passages):
    scores = []
    for passage in passages:
        # Tokenize and prepare input for the model
        inputs = tokenizer(query, passage, return_tensors='pt', truncation=True, padding=True)
        # Get model predictions
        with torch.no_grad():
            outputs = model(**inputs)
        # Extract relevance score (logits)
        score = outputs.logits.squeeze().item()
        scores.append(score)

    # Rank passages based on their scores
    ranked_passages = [(passage, score) for passage, score in sorted(zip(passages, scores), key=lambda x: x[1], reverse=True)]
    return ranked_passages

# Use the retrieved top-k passages from previous step for reranking
top_k_passages = [passage for passage, score in retrieved_passages]
ranked_passages = rank_passages(query, top_k_passages)

# Output the reranked passages and their scores
for passage, score in ranked_passages:
    print(f"Passage: {passage}, Reranked Score: {score}")


tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Passage: in simple terms , inflation is a result of too much money chasing too few goods , i . e . there is an imbalance between demand and supply . the demand exceeds the supply . with all other things being constant it leads to increase in price , i . e . inflation ., Reranked Score: 8.179788589477539
Passage: inflation is a reflection on the expansion of the money supply , aka debt , being created by a central bank . fiat currencies usually inflate , because there is no limit to the amount of debt that can be created . the consequences of reckless money supply expansion can be seen throughout history , see zimbabwe , though there have been many others . . . brazil , argentinia , etc . . ., Reranked Score: 6.900935173034668
Passage: inflation data is a general barometer for inflation that a typical consumer would experience . generally when calculating inflation for yourself you would only include items that you use and in percentages of your budget . personal inflation is much more 

# ***Evaluation***

In [ ]:
from sklearn.metrics import ndcg_score

# Ground truth passage (assuming the correct passage is the last one)
ground_truth = ["Inflation is basically this:  Over time, prices go up! I will now address the 3 points you have listed. Suppose over a period of 10 years, prices have doubled.  Now suppose 10 years ago I earned $100 and bought a nice pair of shoes.  Now today because prices have doubled I would have to earn $200 in order to afford the same pair of shoes.  Thus if I want to compare my earnings this year to 10 years ago, I will need to adjust for the price of goods going up.  That is, I could say that my $100 earnings 10 years ago is the same as having earned $200 today, or alternatively I could say that my earnings of $200 today is equivalent to having earned $100 10 years ago. This is a difficult question because a car is a depreciating asset, which means the real value of the car will go down in value over time. Let us suppose that inflation doesn't exist and the car you bought for $100 today will depreciate to $90 after 1 year (a 10% depreciation). But because inflation does exist, and all prices will be 0.5% higher in 1 years time, we can calculate the true selling price of the car 1 in year as follows: 0.5% of $90 = 0.005*90 = $0.45 Therefore the car will be $90 + $0.45 = $90.45 in 1 years time. If inflation is low, then the repayments do not get much easier to pay back over time because wages have not risen by as much.  Similarly the value of your underlying asset will not increase in value by as much. However as compensation, the interest rates on loans are usually lower when inflation is lower. Therefore generally it is better to get a loan in times of high inflation rather than low inflation, however it really depends on how the much the interest rates are relative to the inflation rate.",
                "When we speak about a product or service, we generally refer to its value. Currency, while neither a product or service, has its own value. As the value of currency goes down, the price of products bought by that currency will go up. You could consider the price of a product or service the value of the product multiplied by the value of the currency. For your first example, we compare two cars, one bought in 1990, and one bought in 2015. Each car has the same features (AC, radio, ABS, etc). We can say that, when these products were new, each had the same value. However, we can deduce that since the 1990 car cost $100, and the 2015 car cost $400, that there has been 75% inflation over 25 years. Comparing prices over time helps identify the inflation (or devaluation of currency) that an economy is experiencing. In regards to your second question, you can say that there was 7% inflation over five years (total). Keep in mind that these are absolute cumulative values. It doesn't mean that there was a 7% increase year over year (that would be 35% inflation over five years), but simply that the absolute value of the dollar has changed 7% over those five years. The sum of the percentages over those five years will be less than 7%, because inflation is measured yearly, but the total cumulative change is 7% from the original value. To put that in perspective, say that you have $100 in 2010, with an expected 7% inflation by 2015, which means that your $100 will be worth $93 in 2015. This means that the yearly inflation would be about 1.5% for five years, resulting in a total of 7% inflation over five years. Note that you still have a hundred dollar bill in your pocket that you've saved for five years, but now that money can buy less product. For example, if you say that $100 buys 50 gallons of gasoline ($2/gallon) in 2010, you will only be able to afford 46.5 gallons with that same bill in 2015 ($2.15/gallon). As you can see, the 7% inflation caused a 7% increase in gasoline prices. In other words, if the value of the car remained the same, its actual price would go up, because the value stayed the same. However, it's more likely that the car's value will decrease significantly in those five years (perhaps as much as 50% or more in some cases), but its price would be higher than it would have been without inflation. If the car's value had dropped 50% (so $50 in original year prices), then it would have a higher price (50 value * 1.07 currency ratio = $53.50). Note that even though its value has decreased by half, its price has not decreased by 50%, because it was hoisted up by inflation. For your final question, the purpose of a loan is so that the loaner will make a profit from the transaction. Consider your prior example where there was 7% inflation over five years. That means that a loan for $100 in 2010 would only be worth $93 in 2015. Interest is how loans combat this loss of value (as well as to earn some profit), so if the loaner expects 7% inflation over five years, they'll charge some higher interest (say 8-10%, or even more), so that when you pay them back on time, they'll come out ahead, or they might use more advanced schemes, like adjustable rates, etc. So, interest rates will naturally be lower when forecasted inflation is lower, and higher when forecasted inflation is higher. The best time to get a loan is when interest rates are low-- if you get locked into a high interest loan and inflation stalls, they will make more money off of you (because the currency has more value), while if inflation skyrockets, your loan will be worth less to loaner. However, they're usually really good about predicting inflation, so it would take an incredible amount of inflation to actually come out on top of a loan.",
                "Inflation refers to the money supply. Think of all money being air in a balloon. Inflation is what happens when you blow more air in the balloon. Deflation is what happens when you let air escape. Inflation may cause prices to go up. However there are many scenarios possible in which this does not happen. For example, at the same time of inflation, there might be unemployment, making consumers unable to pay higher prices. Or some important resource (oil) may go down in price (due to political reasons, war has ended etc), compensating for the money having less value.  Similarly, peoples wages will tend to rise over time. They have to, otherwise everyone would be earning less, due to inflation. However again there are many scenarios in which wages do not keep up with inflation, or rise much faster. In fact over the past 40 years or so, US wages have not been able to keep up with inflation, making the average worker 'poorer' than 40 years ago. At its core, inflation refers to the value of the money itself. As all values of other products, services, assets etc are expressed in terms of money which itself also changes value, this can quickly become very complex. Most countries calculate inflation by averaging the price change of a basket of goods that are supposed to represent the average Joe's spending pattern. However these methods are often criticized as they would be 'hiding' inflation. The hidden inflation may come back later to bite us.",
                "Money itself has no value. A gold bar is worth (fuzzy rushed math, could be totally wrong on this example figure) $423,768.67. So, a 1000 dollars, while worthless paper, are a token saying that you own %.2 of a gold bar in the federal reserve. If a billion dollars are printed, but no new gold is added to the treasury, then your dollar will devalue, and youll only have %.1 percent of that gold bar (again, made up math to describe a hypothetical). When dollars are introduced into the economy, but gold has not been introduced to back it up, things like the government just printing dollars or banks inventing money out of debt (see the housing bubble), then the dollar tokens devalue further. TL;DR: Inflation is the ratio of actual wealth in the Treasury to the amount of currency tokens the treasury has printed.",
                "I've seen a lot of long and complicated answers here so here is my simple and short answer: Let's say the economy consists of: 10 apples and 10$. Then an apple costs 1$. If you print 10$ more you have: 10 apples and 20$. Then an apple costs 2$. That is it! It's not what Kenshin said:  Over time, prices go up! However I would like to add something more on the topic: inflation is theft! If I hack the bank and steal 10% from each account it's obvious that it is theft. It's a bit less obvious when the government prints out money and people loose 10% of the value in their bank accounts but the end result is the same. Final note: some may disagree but I do not consider inflation when 5 of the apples rot and you have: 5 apples and 10$ and an apple now costs 2$. This is a drop in supply and if the demand stays the same prices will rise."]

ground_truth_truncate = truncate_passages(ground_truth)

# List of relevance labels (binary) for all top-k passages
small_embedding_relevance = [1 if p in ground_truth_truncate else 0 for p, _ in retrieved_passages]
small_ranking_relevance = [1 if p in ground_truth_truncate else 0 for p, _ in ranked_passages]

# Calculate NDCG@k for embedding-based and ranking-based results
small_embedding_ndcg = ndcg_score([small_embedding_relevance], [range(len(retrieved_passages))])
small_ranking_ndcg = ndcg_score([small_ranking_relevance], [range(len(ranked_passages))])

print(f"NDCG score for embedding output: {small_embedding_ndcg}")
print(f"NDCG score for ranking output: {small_ranking_ndcg}")

NDCG score for embedding output: 0.28906482631788766
NDCG score for ranking output: 0.3333333333333335


In [ ]:
print(small_ranking_relevance,small_embedding_relevance)

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0] [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
ground_truth_truncate[0]

"inflation is basically this : over time , prices go up ! i will now address the 3 points you have listed . suppose over a period of 10 years , prices have doubled . now suppose 10 years ago i earned $ 100 and bought a nice pair of shoes . now today because prices have doubled i would have to earn $ 200 in order to afford the same pair of shoes . thus if i want to compare my earnings this year to 10 years ago , i will need to adjust for the price of goods going up . that is , i could say that my $ 100 earnings 10 years ago is the same as having earned $ 200 today , or alternatively i could say that my earnings of $ 200 today is equivalent to having earned $ 100 10 years ago . this is a difficult question because a car is a depreciating asset , which means the real value of the car will go down in value over time . let us suppose that inflation doesn ' t exist and the car you bought for $ 100 today will depreciate to $ 90 after 1 year ( a 10 % depreciation ) . but because inflation does

In [ ]:
# Find the index of the row with doc_id 513249
text = df.iloc[(df[df['doc_id'] == 513249].index[0])]['document_text']
text

"Inflation refers to the money supply. Think of all money being air in a balloon. Inflation is what happens when you blow more air in the balloon. Deflation is what happens when you let air escape. Inflation may cause prices to go up. However there are many scenarios possible in which this does not happen. For example, at the same time of inflation, there might be unemployment, making consumers unable to pay higher prices. Or some important resource (oil) may go down in price (due to political reasons, war has ended etc), compensating for the money having less value.  Similarly, peoples wages will tend to rise over time. They have to, otherwise everyone would be earning less, due to inflation. However again there are many scenarios in which wages do not keep up with inflation, or rise much faster. In fact over the past 40 years or so, US wages have not been able to keep up with inflation, making the average worker 'poorer' than 40 years ago. At its core, inflation refers to the value o

In [ ]:
passages[(df[df['doc_id'] == 513249].index[0])]

'there is another aspect too for the high prices of gold . after the current economical crisis people are no more investing in property and a big chunk of investment has been diverted to gold .'

# ***Large embedding Model***

In [ ]:
len(passages[120]),passages[120]

(1218,
 "please consult a tax advisor . you may be voilating the fema [ foreign exchange management act ] and can land into trouble . further what you are doing can land up into various other acts as illegal including aml . further if there is income generated by indian citizen in india , he is still liable to pay tax , irrespective of whether you get the funds back into india . edit : aml is anti money laundering . your transaction is sure to raise aml triggers as it looks like converting black money to white in round about way . once the triggers are raised , rbi division will investigate further to verify what you are doing . if you are able to prove that this is a valid transaction , you would be ok on aml front . how will income tax know ? - if they don ' t know does not mean you are not liable for tax . - any suspicious transactions would get investigated and sooner or later income tax would know about it and can cause a serious problem . - it is irrelevant where you have kept th

In [ ]:
# Define the maximum token length
MAX_TOKEN_LENGTH = 512
# Truncate the document text to the first 512 characters or words
token_512_passages = []
for passage in passages:
    token_512_passages.append(passage[:MAX_TOKEN_LENGTH])

In [ ]:
len(token_512_passages[120]),token_512_passages[120]

(512,
 'please consult a tax advisor . you may be voilating the fema [ foreign exchange management act ] and can land into trouble . further what you are doing can land up into various other acts as illegal including aml . further if there is income generated by indian citizen in india , he is still liable to pay tax , irrespective of whether you get the funds back into india . edit : aml is anti money laundering . your transaction is sure to raise aml triggers as it looks like converting black money to white in ro')

In [ ]:
# Import necessary libraries
import numpy as np
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(
    api_key="secret",  # Replace with your actual API key
    base_url="https://integrate.api.nvidia.com/v1"
)

# Function to embed a list of texts using the large embedding model
def embed_texts(texts, input_type):
    response = client.embeddings.create(
        input=texts,
        model="nvidia/nv-embedqa-e5-v5",
        encoding_format="float",
        extra_body={"input_type": input_type, "truncate": "NONE"}
    )
    return response.data


In [ ]:

# Example usage: Truncate and embed the query and corpus
query_embeddings = embed_texts(query, input_type="query")

import numpy as np

# Set the batch size
BATCH_SIZE = 150
total_passages = len(token_512_passages)
all_corpus_embeddings = []  # To store embeddings for all passages
all_similarity_scores = []  # To store similarity scores across all queries
TOP_K = 10  # Set the number of top passages you want to retrieve

# Function to compute cosine similarity
def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Filter out empty passages
filtered_corpus = [doc for doc in token_512_passages if doc.strip()]

# Process each batch with non-empty passages
for start in range(0, len(filtered_corpus), BATCH_SIZE):
    end = min(start + BATCH_SIZE, len(filtered_corpus))

    # Embed the current batch of corpus passages
    corpus_embeddings_batch = embed_texts(filtered_corpus[start:end], input_type="passage")

    all_corpus_embeddings.extend(corpus_embeddings_batch)  # Store the embeddings

    # Calculate similarity scores for each query in the global list
    for i, query_embedding in enumerate(query_embeddings):
        query_vector = np.array(query_embedding.embedding)

        # Calculate similarity scores for the current batch
        similarity_scores_batch = [cosine_similarity(query_vector, np.array(doc_embedding.embedding)) for doc_embedding in corpus_embeddings_batch]

        # Adjust index for global position and store results in the global similarity list
        for j, score in enumerate(similarity_scores_batch):
            all_similarity_scores.append((start + j, i, score))  # (Global Index, Query Index, Similarity Score)

# Sort the entire similarity list by score in descending order
all_similarity_scores = sorted(all_similarity_scores, key=lambda x: x[2], reverse=True)


In [ ]:

# Example: Retrieve the top 10 overall passages with highest similarity scores
print("Top 10 passages with highest similarity scores:")
for global_index, query_index, score in all_similarity_scores[:10]:
    print(f"Index: {global_index}, Score: {score:.4f}, Passage: {filtered_corpus[global_index][:200]}...")


Top 10 passages with highest similarity scores:
Index: 2365, Score: 0.5156, Passage: inflation is an increase in the money supply . increases in consumer prices follow from inflation . it ' s not the same as inflation . some inflation is necessary for a growing economy . if your gross...
Index: 6262, Score: 0.5021, Passage: in simple terms , inflation is a result of too much money chasing too few goods , i . e . there is an imbalance between demand and supply . the demand exceeds the supply . with all other things being ...
Index: 6665, Score: 0.4929, Passage: inflation refers to the money supply . think of all money being air in a balloon . inflation is what happens when you blow more air in the balloon . deflation is what happens when you let air escape ....
Index: 6663, Score: 0.4899, Passage: inflation is basically this : over time , prices go up ! i will now address the 3 points you have listed . suppose over a period of 10 years , prices have doubled . now suppose 10 years ago i 

In [ ]:
large_model_passages = []

# Store the top 10 passages for reranking
for global_index, query_index, score in all_similarity_scores[:10]:
    large_model_passages.append((filtered_corpus[global_index], score))  # Store passage and score

In [ ]:
large_model_passages[3]

('inflation is basically this : over time , prices go up ! i will now address the 3 points you have listed . suppose over a period of 10 years , prices have doubled . now suppose 10 years ago i earned $ 100 and bought a nice pair of shoes . now today because prices have doubled i would have to earn $ 200 in order to afford the same pair of shoes . thus if i want to compare my earnings this year to 10 years ago , i will need to adjust for the price of goods going up . that is , i could say that my $ 100 earnin',
 0.4898992418528502)

# ***Large Embedding Model + Ranking Model***

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Initialize the tokenizer and model (you can use any ranking model fine-tuned for Q&A tasks)
tokenizer = BertTokenizer.from_pretrained('cross-encoder/ms-marco-MiniLM-L-12-v2')
model = BertForSequenceClassification.from_pretrained('cross-encoder/ms-marco-MiniLM-L-12-v2')

# Function to calculate relevance score using the ranking model
def rank_passages(query, passages):
    scores = []
    for passage in passages:
        # Tokenize and prepare input for the model
        inputs = tokenizer(query, passage, return_tensors='pt', truncation=True, padding=True)
        # Get model predictions
        with torch.no_grad():
            outputs = model(**inputs)
        # Extract relevance score (logits)
        score = outputs.logits.squeeze().item()
        scores.append(score)

    # Rank passages based on their scores
    ranked_passages = [(passage, score) for passage, score in sorted(zip(passages, scores), key=lambda x: x[1], reverse=True)]
    return ranked_passages

# Use the retrieved top-k passages from previous step for reranking
top_k_large_passages = [passage for passage, score in large_model_passages]
large_model_ranked_passages = rank_passages(query, top_k_large_passages)

In [ ]:
# Output the reranked passages and their scores
for passage, score in large_model_ranked_passages:
    print(f"Passage: {passage}, Reranked Score: {score}")

Passage: inflation is an increase in the money supply . increases in consumer prices follow from inflation . it ' s not the same as inflation . some inflation is necessary for a growing economy . if your gross national product is only $ 1 , 000 , then you can get away with having less money than if your gross national product is $ 1 trillion . inflation beyond this , though , is used to allow governments to live beyond their means . if there is more money chasing the same amount of goods , prices will rise . there i, Reranked Score: 8.246162414550781
Passage: in simple terms , inflation is a result of too much money chasing too few goods , i . e . there is an imbalance between demand and supply . the demand exceeds the supply . with all other things being constant it leads to increase in price , i . e . inflation ., Reranked Score: 8.179788589477539
Passage: inflation refers to the money supply . think of all money being air in a balloon . inflation is what happens when you blow more a

# ***Evaluation***

In [ ]:
from sklearn.metrics import ndcg_score

# Truncate the document text to the first 512 characters or words
ground_truth_truncate_512 = []
for truth in ground_truth_truncate:
    ground_truth_truncate_512.append(truth[:MAX_TOKEN_LENGTH])

# List of relevance labels (binary) for all top-k passages
large_embedding_relevance = [1 if p in ground_truth_truncate_512 else 0 for p, _ in large_model_passages]
large_ranking_relevance = [1 if p in ground_truth_truncate_512 else 0 for p, _ in large_model_ranked_passages]

# Calculate NDCG@k for embedding-based and ranking-based results
large_embedding_ndcg = ndcg_score([large_embedding_relevance], [range(len(large_model_passages))])
large_ranking_ndcg = ndcg_score([large_ranking_relevance], [range(len(large_model_ranked_passages))])

print(f"NDCG score for embedding output: {large_embedding_ndcg}")
print(f"NDCG score for ranking output: {large_ranking_ndcg}")

NDCG score for embedding output: 0.3978088012057571
NDCG score for ranking output: 0.5802792108518124


In [ ]:
large_embedding_relevance,large_ranking_relevance

([0, 0, 1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 1, 0])

In [ ]:
ground_truth_truncate_512[0]

'inflation is basically this : over time , prices go up ! i will now address the 3 points you have listed . suppose over a period of 10 years , prices have doubled . now suppose 10 years ago i earned $ 100 and bought a nice pair of shoes . now today because prices have doubled i would have to earn $ 200 in order to afford the same pair of shoes . thus if i want to compare my earnings this year to 10 years ago , i will need to adjust for the price of goods going up . that is , i could say that my $ 100 earnin'

In [ ]:
large_model_passages[3][0]

'inflation is basically this : over time , prices go up ! i will now address the 3 points you have listed . suppose over a period of 10 years , prices have doubled . now suppose 10 years ago i earned $ 100 and bought a nice pair of shoes . now today because prices have doubled i would have to earn $ 200 in order to afford the same pair of shoes . thus if i want to compare my earnings this year to 10 years ago , i will need to adjust for the price of goods going up . that is , i could say that my $ 100 earnin'

In [ ]:
large_model_ranked_passages[8][0]

'inflation is basically this : over time , prices go up ! i will now address the 3 points you have listed . suppose over a period of 10 years , prices have doubled . now suppose 10 years ago i earned $ 100 and bought a nice pair of shoes . now today because prices have doubled i would have to earn $ 200 in order to afford the same pair of shoes . thus if i want to compare my earnings this year to 10 years ago , i will need to adjust for the price of goods going up . that is , i could say that my $ 100 earnin'

# ***Overall Scores***

In [ ]:
import pandas as pd
# Calculate improvements
improvement_small = small_ranking_ndcg - small_embedding_ndcg
improvement_large = large_ranking_ndcg - large_embedding_ndcg

# Create the DataFrame
data = {
    'Embedding Model': [
        'all-MiniLM-L6-v2 (Small)',
        'all-MiniLM-L6-v2 (Small)',
        'nv-embedqa-e5-v5 (Large)',
        'nv-embedqa-e5-v5 (Large)'
    ],
    'Reranking Model': [
        'None',
        'ms-marco-MiniLM-L-12-v2 (ranking)',
        'None',
        'ms-marco-MiniLM-L-12-v2 (ranking)'
    ],
    'NDCG@10 Without Reranking': [
        small_embedding_ndcg,
        small_embedding_ndcg,
        large_embedding_ndcg,
        large_embedding_ndcg
    ],
    'NDCG@10 With Reranking': [
        'N/A',
        small_ranking_ndcg,
        'N/A',
        large_ranking_ndcg
    ],
    'Improvement': [
        'N/A',
        improvement_small,
        'N/A',
        improvement_large
    ]
}

Scores = pd.DataFrame(data)

In [ ]:
Scores

,Embedding Model,Reranking Model,NDCG@10 Without Reranking,NDCG@10 With Reranking,Improvement
0,all-MiniLM-L6-v2 (Small),None,0.289065,N/A,N/A
1,all-MiniLM-L6-v2 (Small),ms-marco-MiniLM-L-12-v2 (ranking),0.289065,0.333333,0.044269
2,nv-embedqa-e5-v5 (Large),None,0.397809,N/A,N/A
3,nv-embedqa-e5-v5 (Large),ms-marco-MiniLM-L-12-v2 (ranking),0.397809,0.580279,0.18247
